# 1.1 Serving a Public Model (GPT-2)
**Protecting Individual's data privacy**

## Dependencies

Make you sure you have [pytorch-transformers](https://github.com/huggingface/pytorch-transformers) installed. PyTorch-Transformers can be installed by pip as follows:

In [1]:
# !pip install pytorch-transformers

In [2]:
import torch as th
import grid as gr
from grid import syft as sy
import gpt2_utils
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

hook = sy.TorchHook(th)

# GPT-2 model
from pytorch_transformers import GPT2LMHeadModel

## Connect to a GridWorker

First, start the grid worker by running the following in a terminal:

```bash
python app/websocket/websocket_app.py --start_local_db --id=app_company --port=3000
```

In [4]:
app_company = gr.WebsocketGridClient(hook, "http://localhost:3000/", id="app_company")
app_company.connect()

## Model and Data Setup

In [ ]:
# Load pre-trained model (weights)
model = GPT2LMHeadModel.from_pretrained('gpt2',torchscript=True)

In [6]:
# Encode a text inputs
text = "Who was Jim Henson ? Jim Henson was a"
tokens_tensor = gpt2_utils.encode_text(text)

IMPORTANT: if you trace the model with a tensor of size `N` the trace model only expects inputs of size `N`.

We need to trace the model in order to make it serializable.

In [8]:
traced_model = th.jit.trace(model, (tokens_tensor,))

/home/marianne/PySyft/syft/generic/frameworks/hook/hook.py:342: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  response = method(*args, **kwargs)
/home/marianne/PySyft/syft/generic/frameworks/hook/hook.py:342: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  response = method(*args, **kwargs)


## Hosting a Public Model

The important part is that we set `allow_download=True` so a user can download a copy of the model to run locally.

In [9]:
app_company.serve_model(traced_model, model_id="GPT-2-public", allow_download=True, allow_remote_inference=False)

True